In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
from terminaltables import AsciiTable
from scipy.stats import chi2

C:\Users\Carlos Abdala\AppData\Local\Temp\ipykernel_16060\3588027038.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
tdf = pd.read_csv('Libro1.csv')
tdf

,Tienda
0,B
1,B
2,B
3,B
4,B
5,B
6,A
7,A
8,B
9,B


In [3]:
tdf = pd.read_csv('Libro1.csv')

current_state_list = []
for i in range(len(tdf) - 1):
    current_state = tdf['Tienda'][i + 1]
    current_state_list.append(current_state)

tdf['EstadoAnterior'] = tdf['Tienda']
tdf = tdf.drop(tdf.index[-1])
tdf['EstadoActual'] = current_state_list
next_state_list=[]
for i in range(len(tdf) - 1):
    next_state = tdf['EstadoActual'][i + 1]
    next_state_list.append(next_state)
tdf = tdf.drop(tdf.index[-1])
tdf['EstadoSiguiente'] = next_state_list
print(tdf)


   Tienda EstadoAnterior EstadoActual EstadoSiguiente
0       B              B            B               B
1       B              B            B               B
2       B              B            B               B
3       B              B            B               B
4       B              B            B               A
5       B              B            A               A
6       A              A            A               B
7       A              A            B               B
8       B              B            B               A
9       B              B            A               B
10      A              A            B               A
11      B              B            A               B
12      A              A            B               B
13      B              B            B               A
14      B              B            A               B


In [4]:
def validate_markov_rule(data):
    
    #PROBABILIDAD OBSERVADA
    states=np.unique(data['EstadoAnterior'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    states_before = list(itertools.product(states,repeat=2))
    states_mapping_before = {}
    for i,state_before in enumerate(states_before):
        complete_state=f"{state_before[0]}{state_before[1]}"
        states_mapping_before[complete_state]=i

    transitions_observed=np.zeros(((len(states_mapping_before)),len(states_mapping)))
    freq_observed=np.zeros(((len(states_mapping_before)),len(states_mapping)))
    for i in range(len(data)):
        freq_observed[states_mapping_before[data.iloc[i]['EstadoAnterior']+data.iloc[i]['EstadoActual']], states_mapping[data.iloc[i]['EstadoSiguiente']]] += 1
    for i in range(len(transitions_observed)):
        transitions_observed[i]=freq_observed[i]/sum(freq_observed[i])
    data_frame_observed=pd.DataFrame(transitions_observed,columns=list(states_mapping.keys()),index=list(states_mapping_before.keys()))
    print(f"Probabilidad Observada \n {data_frame_observed}\n")

    #PROBABILIDAD ESPERADA
    transition_expected = np.zeros(((len(states_mapping)), len(states_mapping)))
    for i in range(len(data)):
        transition_expected[states_mapping[data.iloc[i]['EstadoActual']], states_mapping[data.iloc[i]['EstadoSiguiente']]] += 1
    for i in range(len(transition_expected)):
        transition_expected[i] = transition_expected[i] / sum(transition_expected[i])
    transition_expected=np.tile(transition_expected,(2,1))
    data_frame_expected = pd.DataFrame(transition_expected, columns=list(states_mapping.keys()), index=list(states_mapping.keys())+list(states_mapping.keys()))
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")

    #CHI CUADRADO
    chi2_observed=(np.square(transitions_observed-transition_expected))/transition_expected
    for i in range(len(chi2_observed)):
        chi2_observed[i]=chi2_observed[i]*sum(freq_observed[i])
    chi2_observed=np.sum(chi2_observed)

    #VALORES NECESARIOS
    rows,columns=transitions_observed.shape
    grades=(rows-1)*(columns-1)
    alpha=0.05
    chi2_theory=chi2.ppf(1-alpha,grades)

    print(f"Número de Filas: {rows}")
    print(f"Número de Columnas: {columns}")
    print(f"Grados de Libertad: {grades}")
    print(f"Porcentaje de Confianza: {(1-alpha)*100}%")
    print(f"Chi Cuadrado: {chi2_observed}")
    print(f"Chi Teórico: {chi2_theory}")
    print(f"¿Se Acepta?: {chi2_observed<chi2_theory}")


validate_markov_rule(tdf[['EstadoAnterior', 'EstadoActual', 'EstadoSiguiente']])


Probabilidad Observada 
            A         B
AA  0.000000  1.000000
AB  0.333333  0.666667
BA  0.250000  0.750000
BB  0.428571  0.571429

Probabilidad Esperada
     A    B
A  0.2  0.8
B  0.4  0.6
A  0.2  0.8
B  0.4  0.6

Número de Filas: 4
Número de Columnas: 2
Grados de Libertad: 3
Porcentaje de Confianza: 95.0%
Chi Cuadrado: 0.39186507936507936
Chi Teórico: 7.814727903251179
¿Se Acepta?: True


In [5]:
def markov_model(data):
    #PROBABILIDAD OBSERVADA
    states=np.unique(data['EstadoActual'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #PROBABILIDAD ESPERADA
    transition_matrix = np.zeros(((len(states_mapping)), len(states_mapping)))
    for i in range(len(data)):
        transition_matrix[states_mapping[data.iloc[i]['EstadoActual']], states_mapping[data.iloc[i]['EstadoSiguiente']]] += 1
    for i in range(len(transition_matrix)):
        transition_matrix[i] = transition_matrix[i] / sum(transition_matrix[i])
    data_frame_expected = pd.DataFrame(transition_matrix, columns=list(states_mapping.keys()), index=list(states_mapping.keys()))
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")

    return transition_matrix
#SE ENVIAN LAS COLUMNAS DE ESTADO ACTUAL Y SIGUIENTE
transition_matrix = markov_model(tdf[['EstadoActual', 'EstadoSiguiente']])

print("\nTransitions:")
print(transition_matrix)

Probabilidad Esperada
     A    B
A  0.2  0.8
B  0.4  0.6


Transitions:
[[0.2 0.8]
 [0.4 0.6]]
